# YOLOV4 NESNE TANIMA (OBJECT DETECTİON)
Bu notebookta nesne tanıma (object detection) için en hızlı modellerden biri olan yolov4 modeli ile nesne tanıma yapacağız.

## DARKNET KLONLAMA VE KURULUMU





In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

## YARDIMCI FONKSİYONLAR




In [ ]:
# define helper functions
def imShow(path): 
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

## FOTOĞRAF TOPLAMA VE ETİKETLEME 


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

## HAZIRLAMIŞ OLDUĞUMUZ VERİ SETİNİ YÜKLEME


In [ ]:
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training) 
%cd darknet/
!ls /mydrive/yeni4

In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/yeni4/obj.zip ../
!cp /mydrive/yeni4/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

## EĞİTİM İÇİN GEREKLİ DOSYALAR



*-Config Dosyası*


In [ ]:
# download cfg to google drive and change its name
%cd darknet/
!cp cfg/yolov4-custom.cfg /mydrive/yeni4/yolov4-obj.cfg

[Errno 20] Not a directory: 'darknet/'
/darknet


Config dosyamızda yapmamız gereken değişiklikler:

(Burada verilen değerler bu değişkenlerin önerilen değerleridir.) 

1.   batch = 64 ve subdivision 16.

3.   max_batches değerini (2000 * eğitilen sınıf sayısı) değerine eşitliyoruz.

4.   steps değerlerini (%80 of max_batches) , (%90 of max_batches) yapıyoruz.

5.   [yolo] başlığı altındaki classes değerlerini eğitim yaptığımız sınıf sayısı ile değiştiriyoruz. 

6.   filters değişkenlerini de (eğitim yapacağımız sınıf sayısı + 5 )*3 değerine eşitliyoruz 



In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/yeni4/yolov4-obj.cfg ./cfg

*-obj.names ve obj.data*

yolov4 isimli klasörümüz içine obj.names isimli bir dosya oluşturalım ve dosyayı eğitim yapacağımız nesleriniz isimlerini yazıyoruz.

ÖRNEĞİN:
```
H
```

Aynı klasör içinde obj.data isimle bir dosya oluşturarak içine eğitim yapacağımız nesne sayısını, eğitim yaparken kullanacağımız train.txt, text.txt ve obj.names isimli dosyaların adreslerini ve eğitim sonucu bulduğumuz ağırlıkları kaydedeceğimiz dizini yazıyoruz.

ÖRNEĞİN:
```
classes = 2
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive
```

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yeni4/obj.names ./data
!cp /mydrive/yeni4/obj.data  ./data

*-Train ve Test Dosyaları*


generate_train.py



```
import os
image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")
```

generate_test.py



```
import os

image_files = []
os.chdir(os.path.join("data", "test"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/test/" + filename)
os.chdir("..")
with open("test.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")
```


In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yeni4/generate_train.py ./
!cp /mydrive/yeni4/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

# ÖNCEDEN EĞİTİLMİŞ CONVOLUTİONAL KATMANLARIN AĞIRLIKLARINI İNDİRME

Bu adımda önceden eğitilmiş yolov3 için kullanılmış deeplearning katmanları ağırlıklarını indiriyoruz. Eğitime bu ağırlıklarla başlamak eğittiğimiz modelin daha doğru çalışmasına ve eğitim süresini kısaltmaya yardımcı olacaktır.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

## KONTROL

Gerekli tüm dosyalar hazır, eğitime başlayabiliriz.



Eğitimimiz uzun süreceği için google collab bizi serverdan atabilir. Bunun önüne geçmek için aktif olduğumuzu bir şekile bildirmeliyiz.


Bunun için de sayfanın üst tarafına sağ tıklayıp "ögeyi denetle" veya "incele" seçeneğiniz seçip, çıkan pencereden "console"'a tıklayıp açılan komut satırına aşağıdaki kodu ekleyip enter tuşuna basarsak bu kod bizim 10 dakikada bir connect butonuna basarak bizim aktif kalmamızı sağlayacaktır.

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

# EĞİTİM

Sıradaki komut ile eğitim başlayacaktır.

Eğitimimizin süresi veri setinizdeki fotoğraf sayısı, fotoğrafların kalitesi, eğitim yaptığınız nesne sayısı gibi faktörlere göre değişebilir. Modelimizin doğruluğu için loss değerimiz önemlidir. Loss değerimiz ne kadar düşük olursa modelimiz o kadar doğru çalışır. Modelimizi loss değeri azalmayı durdurana kadar çalıştırıp veri setimize göre mümkün olan en doğru modeli eğitebiliriz. 

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

Modelimizi eğittikten sonra eğitim sırasında loss değerimizin nasıl değiştiğine dair bir grafik görebiliriz.

In [ ]:
# eğitimimize ait grafiğimiz.
imShow('chart.png')

In [ ]:
# eğitime kaldığımız yerden devam edebiliriz.
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/yolov4-obj_last.weights -dont_show -clear

# EĞİTTİĞİMİZ MODELİMİZİ KULLANALIM


Eğitimimiz tamamlandı, şimdi istediğimiz fotoğraflar üzerinde tanıma yapabiliriz.

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yeni4/backup/yolov4-obj_best.weights

# Modelimizi Çalıştıralım


In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/yolov4-obj_last.weights /mydrive/fotograflar/images3.jpg -thresh 0.3 -dont_show
imShow('predictions.jpg')